## TODO:
- if there's not too many illegal chords (on single-voice, there shouldn't be), maybe it's better to throw them away altogether
- maybe add mapping from the resulting chord back to the correctly labeled notes, would probably be much easier

In [27]:
import librosa
import pydub
import numpy as np
import scipy
import os

from pydub import AudioSegment
from pydub.utils import make_chunks
from tqdm.notebook import tqdm, trange

In [2]:
# source: https://www.omnicalculator.com/other/note-frequency#the-note-frequency-chart
freq_table = [[16, 33, 65, 131, 262, 523, 1047, 2093, 4186],\
             [17, 35, 69, 139, 277, 554, 1109, 2217, 4435],\
             [18, 37, 73, 147, 294, 587, 1175, 2349, 4699],\
             [19, 39, 78, 156, 311, 622, 1245, 2489, 4978],\
             [21, 41, 82, 165, 330, 659, 1319, 2637, 5274],\
             [22, 44, 87, 175, 349, 698, 1397, 2794, 5588],\
             [23, 46, 93, 185, 370, 740, 1480, 2960, 5920],\
             [25, 49, 98, 196, 392, 784, 1568, 3136, 6272],\
             [26, 52, 104, 208, 415, 831, 1661, 3322, 6645],\
             [28, 55, 110, 220, 440, 880, 1760, 3520, 7040],\
             [29, 58, 117, 233, 466, 932, 1865, 3729, 7459],\
             [31, 62, 123, 247, 494, 988, 1976, 3951, 7902]]

In [6]:
notes_labels = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
notes = 12
octaves = 9 # the range table covers

max_freq = 8000

In [4]:
err_margins = [0, 1]
curr_margin = 2
for _ in range(octaves - 2):
    err_margins.append(curr_margin)
    curr_margin *= 2
err_margins

[0, 1, 2, 4, 8, 16, 32, 64, 128]

In [7]:
def frequency_to_note(freq):
    if freq < 0 or freq > max_freq:
        return 'invalid'
    for i in range(notes):
        for j in range(octaves):
            if abs(freq - freq_table[i][j]) <= err_margins[j]:
                return notes_labels[i] + str(j)
    return 'unknown'

In [8]:
def are_adjacent_notes(first, second):
    if len(first) > 1:
        first = first[:-1]
    if len(second) > 1:
        second = second[:-1]
    return abs(ord(first) - ord(second)) == 1 or first == 'A' and second == 'G' \
           or first == 'G' and second == 'A' or first == second

In [9]:
def check_chord_valid(chord, note_to_add):
    for note in chord:
        if are_adjacent_notes(note, note_to_add):
            return False
    return True

In [10]:
def get_chord_from_notes(notes):
    chord = []
    curr_index = 0
    while len(chord) < 3 and curr_index < len(notes):
        curr_note = notes[curr_index]
        curr_index += 1
        if curr_note == 'unknown':
            continue
        else:
            curr_note = curr_note[:-1]
        if not curr_note in chord and check_chord_valid(chord, curr_note):
            chord.append(curr_note)
        
    return chord

In [11]:
def transform(samples, sampling_rate):
    n = len(samples)
    t = 1 / sampling_rate
    rb = 1.0 / (2.0 * t)
    yf = scipy.fft.fft(samples)
    xf = np.linspace(0.0, rb, n//2)
    return xf, yf

In [12]:
def get_frequencies(xf, yf):
    n = len(yf)
    found_frequencies = []
    tmp = yf[:n//2]
    for _ in range(len(yf)):
        curr_max_index = np.argmax(tmp)
        found_frequencies.append(xf[curr_max_index])
        tmp = np.delete(tmp, curr_max_index)
        if tmp.size == 0 or xf[curr_max_index] > 8000:
            break
    return found_frequencies

In [13]:
def get_notes(freqs):
    notes = []
    for f in freqs:
        notes.append(frequency_to_note(f))
    return notes

In [26]:
def get_chord_from_sample(filepath):
    samples, sampling_rate = librosa.load(filepath, sr=None, mono=True, offset=0.0, duration=None)
    if len(samples) / sampling_rate > 5:
        chord = ['too long']
        return chord
    xf, yf = transform(samples, sampling_rate)
    freqs = get_frequencies(xf, yf)
    notes = get_notes(freqs)
    chord = get_chord_from_notes(notes)
    return chord
    # fixed = fix_chord(chord)
    # return rename_notes(fixed)

In [16]:
def fix_chord(chord):
    if len(chord) < 3:
        return chord
    chord.sort()
    first = chord[0]
    if len(first) > 1:
        first = first[:-1]
    second = chord[1]
    if len(second) > 1:
        second = second[:-1]
    third = chord[2]
    if len(third) > 1:
        third = third[:-1]
        
    if ord(second) - ord(first) < 2:
        # first is improper
        if ord(third) - ord(second) == 2:
            first = chr(ord(first) - 1)
            chord[0] = first + chord[0][1:]
            return chord
        # second is improper
        elif ord(third) - ord(second) > 2:
            second = chr(ord(second) + 1)
            chord[1] = second + chord[1][1:]
            return chord
        # too fucked up to save, won't even bother
        else:
            return chord
    elif ord(second) - ord(first) > 2:
        # first is improper
        if ord(third) - ord(second) == 2:
            first = chr(ord(first) + 1)
            chord[0] = first + chord[0][1:]
            return chord
        # second is improper
        elif ord(third) - ord(second) < 2:
            second = chr(ord(second) - 1)
            chord[1] = second + chord[1][1:]
            return chord
        # too fucked up to save, won't even bother
        else:
            return chord
        
    if ord(third) - ord(second) > 2:
        # third is improper
        if ord(second) - ord(first) == 2:
            third = chr(ord(third) - 1)
            chord[2] = third + chord[2][1:]
            return chord
        # second is improper
        elif ord(second) - ord(first) < 2:
            second = chr(ord(second) + 1)
            chord[1] = second + chord[1][1:]
            return chord
        # too fucked up to save, won't even bother
        else:
            return chord
    elif ord(third) - ord(second) < 2:
        # third is improper
        if ord(second) - ord(first) == 2:
            third = chr(ord(third) + 1)
            chord[2] = third + chord[2][1:]
            return chord
        # second is improper
        elif ord(second) - ord(first) > 2:
            second = chr(ord(second) - 1)
            chord[1] = second + chord[1][1:]
            return chord
        # too fucked up to save, won't even bother
        else:
            return chord
    return chord

In [20]:
def rename_notes(chord):
    for i in range(3):
        if chord[i] == 'B#':
            chord[i] = 'C'
        if chord[i] == 'E#':
            chord[i] = 'F'
    return chord

In [31]:
def determine_third(root, third):
    root_i = notes_labels.index(root)
    third_i = notes_labels.index(third)
    diff = third_i - root_i
    if diff == 4 or diff == -8:
        return 'major'
    if diff == 3 or diff == -9:
        return 'minor'
    return '!! error !!'

In [24]:
def determine_fifth(root, fifth):
    root_i = notes_labels.index(root)
    fifth_i = notes_labels.index(fifth)
    diff = fifth_i - root_i
    if diff == 7 or diff == -5:
        return 'perfect'
    if diff == 8 or diff == -4:
        return 'aug'
    if abs(diff) == 6:
        return 'dim'
    return '!! error !!'

In [32]:
def determine_chord(notes):
    notes.sort()
    # figure out root note
    # in case it's G
    if ord(notes[2][0]) - ord(notes[1][0]) > 2:
        root_note = notes[2]
        # resort in proper order
        tmp = notes[2]
        notes[2] = notes[0]
        notes[0] = tmp
    else:
        root_note = notes[0]
    # figure out interval for major/minor/etc
    fifth = determine_fifth(notes[0], notes[2])
    third = determine_third(notes[0], notes[1])
    if fifth == 'perfect':
        if third == 'major':
            return root_note
        elif third == 'minor':
            return root_note + 'm'
        else:
            return 'error'
    elif fifth == 'aug':
        if third == 'major':
            return root_note + fifth
        else:
            return 'error'
    elif fifth == 'dim':
        if third == 'minor':
            return root_note + fifth
        else:
            return 'error'
    else:
        return 'error'

In [17]:
def split(path, chunk_duration_ms):
    myaudio = AudioSegment.from_file(path, "wav") 
    chunks = make_chunks(myaudio, chunk_duration_ms)
    chunk_names = []
    for i, chunk in enumerate(chunks): 
        chunk_name = "{0}.wav".format(i) 
        chunk_names.append(chunk_name)
        #print ("exporting", chunk_name) 
        chunk.export(chunk_name, format="wav") 
    return chunk_names

In [37]:
def process_file(path):
    chunks = split(path, 1000)
    chords = []
    for chunk in tqdm(chunks):
        next_chord = get_chord_from_sample(chunk)
        os.remove(chunk)
        print("notes: ", next_chord)
        determined = determine_chord(next_chord)
        print("chord: ", determined)
        chords.append(determined)
    return chords

Example processing a real multi-voice track:

In [38]:
chords = process_file("..\Downloads\\tlsp-eycte.wav")

  0%|          | 0/194 [00:00<?, ?it/s]

notes:  ['C', 'G', 'E']
chord:  C
notes:  ['G#', 'C', 'E']
chord:  Caug
notes:  ['G', 'D', 'B']
chord:  error
notes:  ['G', 'C', 'E']
chord:  C
notes:  ['B', 'G', 'E']
chord:  error
notes:  ['A#', 'D', 'F']
chord:  A#
notes:  ['A#', 'D', 'F#']
chord:  A#aug
notes:  ['C#', 'F', 'A']
chord:  error
notes:  ['G#', 'D', 'B']
chord:  error
notes:  ['D#', 'A#', 'F']
chord:  error
notes:  ['D#', 'F', 'A']
chord:  error
notes:  ['C#', 'A#', 'E']
chord:  A#dim
notes:  ['C#', 'G#', 'E']
chord:  C#m
notes:  ['C', 'F', 'A#']
chord:  error
notes:  ['A#', 'C#', 'F']
chord:  error
notes:  ['A', 'D', 'F#']
chord:  error
notes:  ['G', 'D', 'B']
chord:  error
notes:  ['D#', 'G', 'B']
chord:  error
notes:  ['D#', 'F', 'A#']
chord:  error
notes:  ['F', 'D#', 'A']
chord:  error
notes:  ['C#', 'E', 'A#']
chord:  A#dim
notes:  ['C', 'F', 'A#']
chord:  error
notes:  ['G', 'D', 'B']
chord:  error
notes:  ['A#', 'D', 'F']
chord:  A#
notes:  ['D', 'A#', 'F#']
chord:  A#aug
notes:  ['G#', 'D', 'B']
chord:  error
n

In [39]:
# % of determined legal chords in real track
count = 0
for chord in chords:
    if chord != 'error':
        count += 1
count / len(chords) * 100

41.23711340206185

Example processing samples of guitar chord A:

In [40]:
base_dir = "../Downloads/jim2012Chords/Guitar_Only/a/"
chords_from_samples = []
for i in trange(200):
    next_file = base_dir + "a" + str(i+1) + ".wav"
    next_chord = get_chord_from_sample(next_file)
    determined = determine_chord(next_chord)
    chords_from_samples.append(determined)

  0%|          | 0/200 [00:00<?, ?it/s]

In [41]:
# % of determined legal chords in samples
count = 0
for chord in chords_from_samples:
    if chord != 'error':
        count += 1
count / len(chords) * 100

99.48453608247422

In [42]:
# % of determined correct chords in samples
count = 0
for chord in chords_from_samples:
    if chord == 'A':
        count += 1
count / len(chords) * 100

96.90721649484536